# Query Model Repository for best anomaly detection model

In [25]:
from hops import model
from hops.model import Metric
MODEL_NAME="ganAml"
EVALUATION_METRIC="loss"

In [26]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MIN)

In [27]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: ganAml
Model version: 1
{'loss': '-0.6464197635650635'}

# Create Model Serving of Exported Model

In [28]:
from hops import serving

In [29]:
MODEL_NAME

'ganAml'

In [30]:
# Create serving
model_path="/Models/" + best_model['name']
model_path
response = serving.create_or_update(artifact_path=model_path, serving_name=MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

An error was encountered:
(255, 'Unknown error 255')
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 187, in create_or_update
    artifact_path = hdfs._expand_path(artifact_path)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/hdfs.py", line 190, in _expand_path
    proj_path = project_path(project)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/hdfs.py", line 140, in project_path
    abspath = hdfs.path.abspath("/Projects/" + project + "/")
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/pydoop/hdfs/path.py", line 273, in abspath
    fs = hdfs_fs.hdfs(hostname, port)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/pydoop/hdfs/fs.py", line 166, in __init__
    h, p, u, fs = _get_connection_info(host, port, user)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/pydoop/hdfs/fs.py", line 73, in _get_connection

In [23]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

In [24]:
# Get serving status
serving.get_status(MODEL_NAME)

An error was encountered:
No serving with name: ganAml could be found among the list of available servings: 
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 425, in get_status
    serving = _find_serving_with_name(serving_name, servings)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 466, in _find_serving_with_name
    "available servings: {}".format(serving_name, serving_names_str))
hops.serving.ServingNotFound: No serving with name: ganAml could be found among the list of available servings: 



# Start Model Serving Server

In [9]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

Starting serving with name: ganAml...
Serving with name: ganAml successfully started

In [10]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [18]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

In [19]:
eval_td = fs.get_training_dataset("gan_eval_df", 1)

In [25]:
import numpy as np
eval_td.read().select("embedding").rdd.map(lambda x: np.array(x)[0].tolist()).first()

[0.37102198600769043, -0.09608001261949539, -0.9585465788841248, 0.08975595980882645, 0.4234561026096344, -0.34848344326019287, 1.2133934497833252, 0.3718363344669342, -0.7127293944358826, -0.5090847611427307, 0.5293884873390198, -0.18760253489017487, 0.9332895874977112, -0.5815020203590393]

In [27]:
TOPIC_NAME = serving.get_kafka_topic(MODEL_NAME)
import json

input = [0.37102198600769043, -0.09608001261949539, -0.9585465788841248, 0.08975595980882645, 0.4234561026096344, -0.34848344326019287, 1.2133934497833252, 0.3718363344669342, -0.7127293944358826, -0.5090847611427307, 0.5293884873390198, -0.18760253489017487, 0.9332895874977112, -0.5815020203590393]
data = {
                "signature_name": "serving_default", "instances": [input]
            }
response = serving.make_inference_request(MODEL_NAME, data)
print(response)

An error was encountered:
Could not create or update serving (url: /hopsworks-api/api/project/119/inference/models/ganAml:predict), server response: 
 HTTP code: 400, HTTP reason: Bad Request, error code: 250008, error msg: Serving instance bad request error, user msg: {
    "error": "Serving signature name: \"serving_default\" not found in signature def"
}
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 527, in make_inference_request
    return _make_inference_request_rest(serving_name, data, verb)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/hops/serving.py", line 562, in _make_inference_request_rest
    error_code, error_msg, user_msg))
hops.exceptions.RestAPIError: Could not create or update serving (url: /hopsworks-api/api/project/119/inference/models/ganAml:predict), server response: 
 HTTP code: 400, HTTP reason: Bad Request, error code: 250008, error msg: Serving instance bad requ

In [4]:
import tensorflow as tf
import pydoop.hdfs as pydoop

In [10]:
loaded = tf.saved_model.load(pydoop.path.abspath("hdfs:///Projects/amlsim/Models/ganAml/1"))
print(list(loaded.signatures.keys()))

An error was encountered:
SavedModel file does not exist at: hdfs:///Projects/amlsim/Models/ganAml/1/{saved_model.pbtxt|saved_model.pb}
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/tensorflow/python/saved_model/load.py", line 603, in load
    return load_internal(export_dir, tags, options)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/tensorflow/python/saved_model/load.py", line 614, in load_internal
    loader_impl.parse_saved_model_with_debug_info(export_dir))
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/tensorflow/python/saved_model/loader_impl.py", line 56, in parse_saved_model_with_debug_info
    saved_model = _parse_saved_model(export_dir)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.7/site-packages/tensorflow/python/saved_model/loader_impl.py", line 113, in parse_saved_model
    constants.SAVED_MODEL_FILENAME_PB))
OSError: SavedModel file does not exist at: hdfs:///Projects/